In [24]:
# import
import os
import uuid
import requests
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from IPython.display import display
from geopy.geocoders import Nominatim
import boto3
from IPython.display import display
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker 
from sqlalchemy import Column, Integer, String, Float 
from sqlalchemy.ext.declarative import declarative_base


import json
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

Get weather data with an API

In [2]:
French_Top_Cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre",
                "Rouen", "Paris", "Amiens", "Lille", "Strasbourg",
                "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim",
                "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon",
                "Gorges du Verdon", "Bormes les Mimosas", "Cassis",
                "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes",
                "Aigues Mortes", "Saintes Maries de la mer", "Collioure",
                "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz",
                "Bayonne", "La Rochelle"]


In [3]:
# init dataframe
weather_df = pd.DataFrame(French_Top_Cities, columns=["cities"])
display(weather_df.sample(2))

,cities
29,Ariege
16,Lyon


In [4]:
# Get Cities coordinates 
def cities_to_coords(city_name) :
   
    # geocoder
    geolocator = Nominatim(user_agent="app")
    location = geolocator.geocode(city_name)

    # full_address, latitude, longitude
    full_address = location.address
    latitude = location.latitude
    longitude = location.longitude

    # return 
    return pd.Series([full_address, latitude, longitude])

## RUN
if not "full_address" in weather_df.columns :
    weather_df[["full_address", "latitude", "longitude"]] = weather_df["cities"].apply(cities_to_coords)
    display(weather_df.sample(2))

else : 
    display(weather_df.sample(2))
    print("full_address, latitude & longitude columns exists !")

,cities,full_address,latitude,longitude
34,La Rochelle,"La Rochelle, Charente-Maritime, Nouvelle-Aquit...",46.159113,-1.152043
24,Nimes,"Nîmes, Gard, Occitanie, France métropolitaine,...",43.837425,4.360069


In [5]:
# Get rain volume

# volume rain from oepn weather
def volume_rain(lat, lon, exclude, appid) :
    """ volume of rain in each city over 7 next days.
    """

    # open weather one call api
    params = (
        ('lat', lat),
        ('lon', lon),
        ('exclude', exclude),
        ('appid', appid)
    )

    response = requests.get('https://api.openweathermap.org/data/2.5/onecall', params=params)

    # normalize reponse
    open_weather_7days = pd.json_normalize(response.json()["daily"])

    # clean rain column
    open_weather_7days["rain"] = open_weather_7days["rain"].apply(lambda x : 0 if np.isnan(x) else x) 

    # calculate volumn 
    n_days = open_weather_7days.shape[0]
    volume_rain_7days =  np.sum(open_weather_7days["rain"] * open_weather_7days["pop"]) / n_days

    # return 
    return round(volume_rain_7days,3)



exclude = 'current,minutely,hourly,alerts'
appid = 'b7f01049b4d3e0a7a4251debfe120e99'
    
weather_df["volume_rain_7days"] = weather_df[["latitude", "longitude"]].transpose().apply(lambda x : volume_rain(x[0], x[1], exclude, appid))

display(weather_df.sample(2))

,cities,full_address,latitude,longitude,volume_rain_7days
7,Lille,"Lille, Nord, Hauts-de-France, France métropoli...",50.636565,3.063528,0.369
28,Carcassonne,"Carcassonne, Aude, Occitanie, France métropoli...",43.213036,2.349107,1.823


In [7]:
# unique identifier (uuid)
if not "uuid" in weather_df.columns :
    weather_df['uuid'] = weather_df.index.to_series().map(lambda x: uuid.uuid4())

display(weather_df.sample(2))

,cities,full_address,latitude,longitude,volume_rain_7days,uuid
8,Strasbourg,"Strasbourg, Bas-Rhin, Grand Est, France métrop...",48.584614,7.750713,0.835,b7c7351c-d2af-47d5-bdbc-01b69fb1811a
20,Marseille,"Marseille, Bouches-du-Rhône, Provence-Alpes-Cô...",43.296174,5.369953,0.822,7299bf86-4d9c-4b0f-abde-f2ed065c59b2


In [8]:
# reorder columns
keep_col = ['uuid', 'cities', 'full_address', 'latitude', 'longitude', 'volume_rain_7days']
weather_df = weather_df[keep_col]

display(weather_df.sample(2))

,uuid,cities,full_address,latitude,longitude,volume_rain_7days
4,0bcc9f24-7709-4986-a122-821f945bc072,Rouen,"Rouen, Seine-Maritime, Normandie, France métro...",49.440459,1.093966,0.435
7,076f7ea8-a1fd-42df-8e12-d47646b22d54,Lille,"Lille, Nord, Hauts-de-France, France métropoli...",50.636565,3.063528,0.369


In [9]:
# Save dataframe
weather_data = "data/weather_data.csv"
overwrite = False 

if not os.path.exists(weather_data) or overwrite==True:
    weather_df.to_csv(weather_data)

else :
    print(f"{weather_data} exists !")

data_2/weather_data.csv exists !


In [10]:
# fetch best destinations
top_5_destinations = weather_df.sort_values(by=['volume_rain_7days'])[:5]
top_20_destinations = weather_df.sort_values(by=['volume_rain_7days'])[:20]
display(top_5_destinations.sample(2))
display(top_20_destinations.sample(2))


,uuid,cities,full_address,latitude,longitude,volume_rain_7days
6,f15c2902-4e29-4077-aa83-e02d625f759d,Amiens,"Amiens, Somme, Hauts-de-France, France métropo...",49.894171,2.295695,0.252
7,076f7ea8-a1fd-42df-8e12-d47646b22d54,Lille,"Lille, Nord, Hauts-de-France, France métropoli...",50.636565,3.063528,0.369


,uuid,cities,full_address,latitude,longitude,volume_rain_7days
20,7299bf86-4d9c-4b0f-abde-f2ed065c59b2,Marseille,"Marseille, Bouches-du-Rhône, Provence-Alpes-Cô...",43.296174,5.369953,0.822
11,67314675-4ef2-4958-b2a2-2d1c3b796c41,Eguisheim,"Eguisheim, Colmar-Ribeauvillé, Haut-Rhin, Gran...",48.044797,7.307962,1.388


In [13]:
    # fig
lat_col = top_5_destinations['latitude']
lon_col = top_5_destinations['longitude']
color_col = 'red'
size_col= top_5_destinations['volume_rain_7days']

fig = px.scatter_mapbox(top_5_destinations, lat=lat_col, lon=lon_col,     
                        color=size_col, size=size_col,
                        color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=15)

fig.show()

Scrape Booking.com

In [18]:
class HotelsSpider(scrapy.Spider):
    name = "hotels"
    def start_requests(self):

        # first page by city
        urls = [
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bclass_interval%3D1%3Bdest_id%3D-1413801%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bnflt%3Dht_id%253D204%3Bno_rooms%3D1%3Boffset%3D25%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dindex%3Bsrpvid%3De2c56fbc232b004e%3Bss%3DBormes-les-Mimosas%252C%2520Provence-Alpes-C%25C3%25B4te%2520d%2527Azur%252C%2520France%3Bss_all%3D0%3Bss_raw%3Dbormes%3Bssb%3Dempty%3Bsshis%3D0%3Btop_ufis%3D1%26%3B&ss=Le+Mont-Saint-Michel%2C+Basse-Normandie%2C+France&is_ski_area=&ssne=Bormes-les-Mimosas&ssne_untouched=Bormes-les-Mimosas&city=-1413801&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&ss_raw=Mont+Saint+Michel&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=900039327&dest_type=city&place_id_lat=48.636&place_id_lon=-1.511&search_pageview_id=e2c56fbc232b004e&search_selected=true&search_pageview_id=e2c56fbc232b004e&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1413801%3Bclass_interval%3D1%3Bdest_id%3D900039327%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3Db609813456090076%3Bss%3DLe%2520Mont-Saint-Michel%252C%2520Basse-Normandie%252C%2520France%3Bss_all%3D0%3Bss_raw%3DMont%2520Saint%2520Michel%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DBormes-les-Mimosas%3Bssne_untouched%3DBormes-les-Mimosas%3Btop_ufis%3D1%26%3B&ss=Saint-Malo%2C+Bretagne%2C+France&is_ski_area=&ssne=Le+Mont-Saint-Michel&ssne_untouched=Le+Mont-Saint-Michel&city=900039327&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&ss_raw=St+Malo&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1466824&dest_type=city&place_id_lat=48.649616&place_id_lon=-2.025677&search_pageview_id=b609813456090076&search_selected=true&search_pageview_id=b609813456090076&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D900039327%3Bclass_interval%3D1%3Bdest_id%3D-1466824%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3Dc635814e1d6f00b5%3Bss%3DSaint-Malo%252C%2520Bretagne%252C%2520France%3Bss_all%3D0%3Bss_raw%3DSt%2520Malo%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DLe%2520Mont-Saint-Michel%3Bssne_untouched%3DLe%2520Mont-Saint-Michel%3Btop_ufis%3D1%26%3B&ss=Bayeux&is_ski_area=0&ssne=Saint-Malo&ssne_untouched=Saint-Malo&city=-1466824&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&ss_raw=Bayeux&search_pageview_id=c635814e1d6f00b5&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bcity%3D-1466824%3Bclass_interval%3D1%3Bdest_id%3D-1466824%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_pageview_id%3Dc635814e1d6f00b5%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D058e81590c390178%3Bss%3DBayeux%3Bss_all%3D0%3Bss_raw%3DBayeux%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DSaint-Malo%3Bssne_untouched%3DSaint-Malo%3Btop_ufis%3D1%26%3B&ss=Le+Havre%2C+Haute-Normandie%2C+France&is_ski_area=&ssne=Saint-Malo&ssne_untouched=Saint-Malo&city=-1466824&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&ss_raw=Le+Havre&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1441598&dest_type=city&place_id_lat=49.49489&place_id_lon=0.107723&search_pageview_id=058e81590c390178&search_selected=true&search_pageview_id=058e81590c390178&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1466824%3Bclass_interval%3D1%3Bdest_id%3D-1441598%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D74678168e1e70029%3Bss%3DLe%2520Havre%252C%2520Haute-Normandie%252C%2520France%3Bss_all%3D0%3Bss_raw%3DLe%2520Havre%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DSaint-Malo%3Bssne_untouched%3DSaint-Malo%3Btop_ufis%3D1%26%3B&ss=Rouen%2C+Haute-Normandie%2C+France&is_ski_area=&ssne=Le+Havre&ssne_untouched=Le+Havre&city=-1441598&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&ss_raw=Rouen&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1462807&dest_type=city&place_id_lat=49.44046&place_id_lon=1.09397&search_pageview_id=74678168e1e70029&search_selected=true&search_pageview_id=74678168e1e70029&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1441598%3Bclass_interval%3D1%3Bdest_id%3D-1462807%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D9c39817b30540031%3Bss%3DRouen%252C%2520Haute-Normandie%252C%2520France%3Bss_all%3D0%3Bss_raw%3DRouen%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DLe%2520Havre%3Bssne_untouched%3DLe%2520Havre%3Btop_ufis%3D1%26%3B&ss=Paris%2C+%C3%8Ele-de-France%2C+France&is_ski_area=0&ssne=Rouen&ssne_untouched=Rouen&city=-1462807&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&ss_raw=Paris&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1456928&dest_type=city&iata=PAR&place_id_lat=48.85668&place_id_lon=2.351476&search_pageview_id=9c39817b30540031&search_selected=true&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bcity%3D-1456928%3Bclass_interval%3D1%3Bdest_id%3D-1456928%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_pageview_id%3D96838187422b0181%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D44fb81931f11015b%3Bss%3DAmiens%3Bss_all%3D0%3Bss_raw%3DAmiens%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DParis%3Bssne_untouched%3DParis%3Btop_ufis%3D1%26%3B&ss=Amiens%2C+Picardie%2C+France&is_ski_area=&ssne=Paris&ssne_untouched=Paris&city=-1456928&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&ss_raw=Amiens&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1407447&dest_type=city&place_id_lat=49.895454&place_id_lon=2.299816&search_pageview_id=44fb81931f11015b&search_selected=true&search_pageview_id=44fb81931f11015b&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1456928%3Bclass_interval%3D1%3Bdest_id%3D-1407447%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D19cc8198cf82016e%3Bss%3DAmiens%252C%2520Picardie%252C%2520France%3Bss_all%3D0%3Bss_raw%3DAmiens%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DParis%3Bssne_untouched%3DParis%3Btop_ufis%3D1%26%3B&ss=Lille%2C+Nord-Pas-de-Calais%2C+France&is_ski_area=&ssne=Amiens&ssne_untouched=Amiens&city=-1407447&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&ss_raw=Lille&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1447079&dest_type=city&iata=LIL&place_id_lat=50.636845&place_id_lon=3.063454&search_pageview_id=19cc8198cf82016e&search_selected=true&search_pageview_id=19cc8198cf82016e&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1407447%3Bclass_interval%3D1%3Bdest_id%3D-1447079%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Biata%3DLIL%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3Deb1981a576570052%3Bss%3DLille%252C%2520Nord-Pas-de-Calais%252C%2520France%3Bss_all%3D0%3Bss_raw%3DLille%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DAmiens%3Bssne_untouched%3DAmiens%3Btop_ufis%3D1%26%3B&ss=Strasbourg%2C+Alsace%2C+France&is_ski_area=0&ssne=Lille&ssne_untouched=Lille&city=-1447079&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1471697&dest_type=city&iata=SXB&place_id_lat=48.5834&place_id_lon=7.7458&search_pageview_id=eb1981a576570052&search_selected=true&ss_raw=Strasbourg&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1447079%3Bclass_interval%3D1%3Bdest_id%3D-1471697%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Biata%3DSXB%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3Dd16681b280140136%3Bss%3DStrasbourg%252C%2520Alsace%252C%2520France%3Bss_all%3D0%3Bss_raw%3DStrasbourg%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DLille%3Bssne_untouched%3DLille%3Btop_ufis%3D1%26%3B&ss=Ch%C3%A2teau+du+Haut-K%C5%93nigsbourg%2C+Saint-Hippolyte%2C+Alsace%2C+France&is_ski_area=&ssne=Strasbourg&ssne_untouched=Strasbourg&city=-1471697&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=d16681b280140136&ac_suggestion_list_length=3&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=204055&dest_type=landmark&place_id_lat=48.24931&place_id_lon=7.34445&search_pageview_id=d16681b280140136&search_selected=true&ss_raw=Chateau+du+Haut+Koenigsbourg&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1471697%3Bclass_interval%3D1%3Bdest_id%3D204055%3Bdest_type%3Dlandmark%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dlandmark%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D119981c3883200d7%3Bss%3DCh%25C3%25A2teau%2520du%2520Haut-K%25C5%2593nigsbourg%252C%2520Saint-Hippolyte%252C%2520Alsace%252C%2520France%3Bss_all%3D0%3Bss_raw%3DChateau%2520du%2520Haut%2520Koenigsbourg%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DStrasbourg%3Bssne_untouched%3DStrasbourg%3Btop_ufis%3D1%26%3B&ss=Colmar%2C+Alsace%2C+France&is_ski_area=&ssne=Ch%C3%A2teau+du+Haut-K%C5%93nigsbourg&ssne_untouched=Ch%C3%A2teau+du+Haut-K%C5%93nigsbourg&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&ss_raw=Colmar&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1421049&dest_type=city&place_id_lat=48.079155&place_id_lon=7.357718&search_pageview_id=119981c3883200d7&search_selected=true&search_pageview_id=119981c3883200d7&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bclass_interval%3D1%3Bdest_id%3D-1421049%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3Da96481d31d3c0147%3Bss%3DColmar%252C%2520Alsace%252C%2520France%3Bss_all%3D0%3Bss_raw%3DColmar%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DCh%25C3%25A2teau%2520du%2520Haut-K%25C5%2593nigsbourg%3Bssne_untouched%3DCh%25C3%25A2teau%2520du%2520Haut-K%25C5%2593nigsbourg%3Btop_ufis%3D1%26%3B&ss=Eguisheim%2C+Alsace%2C+France&is_ski_area=&ssne=Colmar&ssne_untouched=Colmar&city=-1421049&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=a96481d31d3c0147&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1425030&dest_type=city&place_id_lat=48.04231&place_id_lon=7.30543&search_pageview_id=a96481d31d3c0147&search_selected=true&ss_raw=Eguisheim&nflt=health_and_hygiene%3D1',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1421049%3Bclass_interval%3D1%3Bdest_id%3D-1425030%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D783681e176630022%3Bss%3DEguisheim%252C%2520Alsace%252C%2520France%3Bss_all%3D0%3Bss_raw%3DEguisheim%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DColmar%3Bssne_untouched%3DColmar%3Btop_ufis%3D1%26%3B&ss=Besan%C3%A7on%2C+Franche-Comt%C3%A9%2C+France&is_ski_area=&ssne=Eguisheim&ssne_untouched=Eguisheim&city=-1425030&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=783681e176630022&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1412198&dest_type=city&place_id_lat=47.240215&place_id_lon=6.023773&search_pageview_id=783681e176630022&search_selected=true&ss_raw=Besancon&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1425030%3Bclass_interval%3D1%3Bdest_id%3D-1412198%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D17b581f1c397017a%3Bss%3DBesan%25C3%25A7on%252C%2520Franche-Comt%25C3%25A9%252C%2520France%3Bss_all%3D0%3Bss_raw%3DBesancon%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DEguisheim%3Bssne_untouched%3DEguisheim%3Btop_ufis%3D1%26%3B&ss=Dijon%2C+Bourgogne%2C+France&is_ski_area=&ssne=Besan%C3%A7on&ssne_untouched=Besan%C3%A7on&city=-1412198&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=17b581f1c397017a&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1423981&dest_type=city&place_id_lat=47.321762&place_id_lon=5.039809&search_pageview_id=17b581f1c397017a&search_selected=true&ss_raw=Dijon&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1412198%3Bclass_interval%3D1%3Bdest_id%3D-1423981%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3Da332820097fb0091%3Bss%3DDijon%252C%2520Bourgogne%252C%2520France%3Bss_all%3D0%3Bss_raw%3DDijon%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DBesan%25C3%25A7on%3Bssne_untouched%3DBesan%25C3%25A7on%3Btop_ufis%3D1%26%3B&ss=Annecy%2C+Rh%C3%B4ne-Alpes%2C+France&is_ski_area=&ssne=Dijon&ssne_untouched=Dijon&city=-1423981&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=a332820097fb0091&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1407760&dest_type=city&iata=NCY&place_id_lat=45.899727&place_id_lon=6.127196&search_pageview_id=a332820097fb0091&search_selected=true&ss_raw=Annecy&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1423981%3Bclass_interval%3D1%3Bdest_id%3D-1407760%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Biata%3DNCY%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D3a96820d7681016d%3Bss%3DAnnecy%252C%2520Rh%25C3%25B4ne-Alpes%252C%2520France%3Bss_all%3D0%3Bss_raw%3DAnnecy%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DDijon%3Bssne_untouched%3DDijon%3Btop_ufis%3D1%26%3B&ss=Grenoble%2C+Rh%C3%B4ne-Alpes%2C+France&is_ski_area=&ssne=Annecy&ssne_untouched=Annecy&city=-1407760&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=3a96820d7681016d&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1430647&dest_type=city&place_id_lat=45.192505&place_id_lon=5.724973&search_pageview_id=3a96820d7681016d&search_selected=true&ss_raw=Grenoble&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1407760%3Bclass_interval%3D1%3Bdest_id%3D-1430647%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3Db13d821e522a01fd%3Bss%3DGrenoble%252C%2520Rh%25C3%25B4ne-Alpes%252C%2520France%3Bss_all%3D0%3Bss_raw%3DGrenoble%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DAnnecy%3Bssne_untouched%3DAnnecy%3Btop_ufis%3D1%26%3B&ss=Lyon%2C+Rh%C3%B4ne-Alpes%2C+France&is_ski_area=&ssne=Grenoble&ssne_untouched=Grenoble&city=-1430647&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=b13d821e522a01fd&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1448468&dest_type=city&iata=LYS&place_id_lat=45.757767&place_id_lon=4.832204&search_pageview_id=b13d821e522a01fd&search_selected=true&ss_raw=Lyon&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1430647%3Bclass_interval%3D1%3Bdest_id%3D-1448468%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Biata%3DLYS%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D71a0822b130200f7%3Bss%3DLyon%252C%2520Rh%25C3%25B4ne-Alpes%252C%2520France%3Bss_all%3D0%3Bss_raw%3DLyon%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DGrenoble%3Bssne_untouched%3DGrenoble%3Btop_ufis%3D1%26%3B&ss=Gorges+du+Verdon%2C+France&is_ski_area=&ssne=Lyon&ssne_untouched=Lyon&city=-1448468&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&ss_raw=Gorges+du+Verdon&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=2746&dest_type=region&place_id_lat=43.78571&place_id_lon=6.127711&search_pageview_id=71a0822b130200f7&search_selected=true&search_pageview_id=71a0822b130200f7&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1448468%3Bclass_interval%3D1%3Bdest_id%3D2746%3Bdest_type%3Dregion%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dregion%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D65a68243dd260133%3Bss%3DGorges%2520du%2520Verdon%252C%2520France%3Bss_all%3D0%3Bss_raw%3DGorges%2520du%2520Verdon%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DLyon%3Bssne_untouched%3DLyon%3Btop_ufis%3D1%26%3B&ss=Bormes-les-Mimosas%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=&ssne=Gorges+du+Verdon&ssne_untouched=Gorges+du+Verdon&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=65a68243dd260133&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1413801&dest_type=city&place_id_lat=43.1507&place_id_lon=6.34193&search_pageview_id=65a68243dd260133&search_selected=true&ss_raw=Bormes+les+Mimosas&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bclass_interval%3D1%3Bdest_id%3D-1413801%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D359f8277033700cb%3Bss%3DBormes-les-Mimosas%252C%2520Provence-Alpes-C%25C3%25B4te%2520d%2527Azur%252C%2520France%3Bss_all%3D0%3Bss_raw%3DBormes%2520les%2520Mimosas%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DGorges%2520du%2520Verdon%3Bssne_untouched%3DGorges%2520du%2520Verdon%3Btop_ufis%3D1%26%3B&ss=Cassis%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=&ssne=Bormes-les-Mimosas&ssne_untouched=Bormes-les-Mimosas&city=-1413801&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=359f8277033700cb&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1416912&dest_type=city&place_id_lat=43.21513&place_id_lon=5.53712&search_pageview_id=359f8277033700cb&search_selected=true&ss_raw=Cassis&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1413801%3Bclass_interval%3D1%3Bdest_id%3D-1416912%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D65e58297e63e0056%3Bss%3DCassis%252C%2520Provence-Alpes-C%25C3%25B4te%2520d%2527Azur%252C%2520France%3Bss_all%3D0%3Bss_raw%3DCassis%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DBormes-les-Mimosas%3Bssne_untouched%3DBormes-les-Mimosas%3Btop_ufis%3D1%26%3B&ss=Marseille%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=&ssne=Cassis&ssne_untouched=Cassis&city=-1416912&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&ss_raw=Marseille&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1449947&dest_type=city&iata=MRS&place_id_lat=43.2983&place_id_lon=5.37798&search_pageview_id=65e58297e63e0056&search_selected=true&search_pageview_id=65e58297e63e0056&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1416912%3Bclass_interval%3D1%3Bdest_id%3D-1449947%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Biata%3DMRS%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D1f2d82a4c76b01df%3Bss%3DMarseille%252C%2520Provence-Alpes-C%25C3%25B4te%2520d%2527Azur%252C%2520France%3Bss_all%3D0%3Bss_raw%3DMarseille%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DCassis%3Bssne_untouched%3DCassis%3Btop_ufis%3D1%26%3B&ss=Aix-en-Provence%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=&ssne=Marseille&ssne_untouched=Marseille&city=-1449947&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&ss_raw=Aix+en+Provence&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1406939&dest_type=city&place_id_lat=43.526314&place_id_lon=5.445464&search_pageview_id=1f2d82a4c76b01df&search_selected=true&search_pageview_id=1f2d82a4c76b01df&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1449947%3Bclass_interval%3D1%3Bdest_id%3D-1406939%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D18b182ba2a9f0111%3Bss%3DAix-en-Provence%252C%2520Provence-Alpes-C%25C3%25B4te%2520d%2527Azur%252C%2520France%3Bss_all%3D0%3Bss_raw%3DAix%2520en%2520Provence%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DMarseille%3Bssne_untouched%3DMarseille%3Btop_ufis%3D1%26%3B&ss=Avignon%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=&ssne=Aix-en-Provence&ssne_untouched=Aix-en-Provence&city=-1406939&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=18b182ba2a9f0111&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1409631&dest_type=city&iata=AVN&place_id_lat=43.949&place_id_lon=4.807508&search_pageview_id=18b182ba2a9f0111&search_selected=true&ss_raw=Avignon&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1406939%3Bclass_interval%3D1%3Bdest_id%3D-1409631%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Biata%3DAVN%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3Deb1c82c6d060014c%3Bss%3DAvignon%252C%2520Provence-Alpes-C%25C3%25B4te%2520d%2527Azur%252C%2520France%3Bss_all%3D0%3Bss_raw%3DAvignon%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DAix-en-Provence%3Bssne_untouched%3DAix-en-Provence%3Btop_ufis%3D1%26%3B&ss=Uz%C3%A8s%2C+Languedoc-Roussillon%2C+France&is_ski_area=&ssne=Avignon&ssne_untouched=Avignon&city=-1409631&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=eb1c82c6d060014c&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1474231&dest_type=city&place_id_lat=44.01217&place_id_lon=4.41967&search_pageview_id=eb1c82c6d060014c&search_selected=true&ss_raw=Uzes&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1409631%3Bclass_interval%3D1%3Bdest_id%3D-1474231%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D3c9282d968fe002b%3Bss%3DUz%25C3%25A8s%252C%2520Languedoc-Roussillon%252C%2520France%3Bss_all%3D0%3Bss_raw%3DUzes%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DAvignon%3Bssne_untouched%3DAvignon%3Btop_ufis%3D1%26%3B&ss=N%C3%AEmes%2C+Languedoc-Roussillon%2C+France&is_ski_area=&ssne=Uz%C3%A8s&ssne_untouched=Uz%C3%A8s&city=-1474231&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=3c9282d968fe002b&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1455068&dest_type=city&iata=FNI&place_id_lat=43.83742&place_id_lon=4.36007&search_pageview_id=3c9282d968fe002b&search_selected=true&ss_raw=Nimes&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1474231%3Bclass_interval%3D1%3Bdest_id%3D-1455068%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Biata%3DFNI%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3Daf0c82e7fdec019d%3Bss%3DN%25C3%25AEmes%252C%2520Languedoc-Roussillon%252C%2520France%3Bss_all%3D0%3Bss_raw%3DNimes%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DUz%25C3%25A8s%3Bssne_untouched%3DUz%25C3%25A8s%3Btop_ufis%3D1%26%3B&ss=Aigues-Mortes%2C+Languedoc-Roussillon%2C+France&is_ski_area=&ssne=N%C3%AEmes&ssne_untouched=N%C3%AEmes&city=-1455068&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=af0c82e7fdec019d&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1406800&dest_type=city&place_id_lat=43.566887&place_id_lon=4.190193&search_pageview_id=af0c82e7fdec019d&search_selected=true&ss_raw=Aigues+Mortes&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1455068%3Bclass_interval%3D1%3Bdest_id%3D-1406800%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3Dd16682fbc3b10014%3Bss%3DAigues-Mortes%252C%2520Languedoc-Roussillon%252C%2520France%3Bss_all%3D0%3Bss_raw%3DAigues%2520Mortes%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DN%25C3%25AEmes%3Bssne_untouched%3DN%25C3%25AEmes%3Btop_ufis%3D1%26%3B&ss=Les+Saintes-Maries-de-la-Mer%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=&ssne=Aigues-Mortes&ssne_untouched=Aigues-Mortes&city=-1406800&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=d16682fbc3b10014&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1465138&dest_type=city&place_id_lat=43.45233&place_id_lon=4.42871&search_pageview_id=d16682fbc3b10014&search_selected=true&ss_raw=Saintes+Maries+de+la+mer&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1406800%3Bclass_interval%3D1%3Bdest_id%3D-1465138%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D9d91830a74190103%3Bss%3DLes%2520Saintes-Maries-de-la-Mer%252C%2520Provence-Alpes-C%25C3%25B4te%2520d%2527Azur%252C%2520France%3Bss_all%3D0%3Bss_raw%3DSaintes%2520Maries%2520de%2520la%2520mer%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DAigues-Mortes%3Bssne_untouched%3DAigues-Mortes%3Btop_ufis%3D1%26%3B&ss=Collioure%2C+Languedoc-Roussillon%2C+France&is_ski_area=&ssne=Les+Saintes-Maries-de-la-Mer&ssne_untouched=Les+Saintes-Maries-de-la-Mer&city=-1465138&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=9d91830a74190103&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1421032&dest_type=city&place_id_lat=42.52453&place_id_lon=3.082255&search_pageview_id=9d91830a74190103&search_selected=true&ss_raw=Collioure&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1465138%3Bclass_interval%3D1%3Bdest_id%3D-1421032%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D5c798315651700c3%3Bss%3DCollioure%252C%2520Languedoc-Roussillon%252C%2520France%3Bss_all%3D0%3Bss_raw%3DCollioure%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DLes%2520Saintes-Maries-de-la-Mer%3Bssne_untouched%3DLes%2520Saintes-Maries-de-la-Mer%3Btop_ufis%3D1%26%3B&ss=Carcassonne%2C+Languedoc-Roussillon%2C+France&is_ski_area=&ssne=Collioure&ssne_untouched=Collioure&city=-1421032&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=5c798315651700c3&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1416701&dest_type=city&iata=CCF&place_id_lat=43.21304&place_id_lon=2.34911&search_pageview_id=5c798315651700c3&search_selected=true&ss_raw=Carcassonne&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1421032%3Bclass_interval%3D1%3Bdest_id%3D-1416701%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Biata%3DCCF%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3Db12b8324d8fb009a%3Bss%3DCarcassonne%252C%2520Languedoc-Roussillon%252C%2520France%3Bss_all%3D0%3Bss_raw%3DCarcassonne%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DCollioure%3Bssne_untouched%3DCollioure%3Btop_ufis%3D1%26%3B&ss=Ari%C3%A8ge%2C+France&is_ski_area=&ssne=Carcassonne&ssne_untouched=Carcassonne&city=-1416701&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=b12b8324d8fb009a&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=2507&dest_type=region&place_id_lat=42.89857&place_id_lon=1.612595&search_pageview_id=b12b8324d8fb009a&search_selected=true&ss_raw=Ariege&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1416701%3Bclass_interval%3D1%3Bdest_id%3D2507%3Bdest_type%3Dregion%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dregion%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D8e3d8331e7670002%3Bss%3DAri%25C3%25A8ge%252C%2520France%3Bss_all%3D0%3Bss_raw%3DAriege%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DCarcassonne%3Bssne_untouched%3DCarcassonne%3Btop_ufis%3D1%26%3B&ss=Toulouse%2C+Midi-Pyr%C3%A9n%C3%A9es%2C+France&is_ski_area=&ssne=Ari%C3%A8ge&ssne_untouched=Ari%C3%A8ge&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=8e3d8331e7670002&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1473166&dest_type=city&iata=TLS&place_id_lat=43.604645&place_id_lon=1.444213&search_pageview_id=8e3d8331e7670002&search_selected=true&ss_raw=Toulouse&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bclass_interval%3D1%3Bdest_id%3D-1473166%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Biata%3DTLS%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D45f8833e955e00be%3Bss%3DToulouse%252C%2520Midi-Pyr%25C3%25A9n%25C3%25A9es%252C%2520France%3Bss_all%3D0%3Bss_raw%3DToulouse%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DAri%25C3%25A8ge%3Bssne_untouched%3DAri%25C3%25A8ge%3Btop_ufis%3D1%26%3B&ss=Montauban%2C+Midi-Pyr%C3%A9n%C3%A9es%2C+France&is_ski_area=&ssne=Toulouse&ssne_untouched=Toulouse&city=-1473166&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=45f8833e955e00be&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1452421&dest_type=city&place_id_lat=44.01758&place_id_lon=1.355&search_pageview_id=45f8833e955e00be&search_selected=true&ss_raw=Montauban&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1473166%3Bclass_interval%3D1%3Bdest_id%3D-1452421%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D9cb18360e9040099%3Bss%3DMontauban%252C%2520Midi-Pyr%25C3%25A9n%25C3%25A9es%252C%2520France%3Bss_all%3D0%3Bss_raw%3DMontauban%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DToulouse%3Bssne_untouched%3DToulouse%3Btop_ufis%3D1%26%3B&ss=Biarritz%2C+Aquitaine%2C+France&is_ski_area=&ssne=Montauban&ssne_untouched=Montauban&city=-1452421&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=9cb18360e9040099&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1412526&dest_type=city&iata=BIQ&place_id_lat=43.4804&place_id_lon=-1.56066&search_pageview_id=9cb18360e9040099&search_selected=true&ss_raw=Biarritz&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1452421%3Bclass_interval%3D1%3Bdest_id%3D-1412526%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Biata%3DBIQ%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3Df2958372b60201d3%3Bss%3DBiarritz%252C%2520Aquitaine%252C%2520France%3Bss_all%3D0%3Bss_raw%3DBiarritz%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DMontauban%3Bssne_untouched%3DMontauban%3Btop_ufis%3D1%26%3B&ss=Bayonne%2C+Aquitaine%2C+France&is_ski_area=&ssne=Biarritz&ssne_untouched=Biarritz&city=-1412526&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=f2958372b60201d3&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1410844&dest_type=city&place_id_lat=43.49334&place_id_lon=-1.4751&search_pageview_id=f2958372b60201d3&search_selected=true&ss_raw=Bayonne&nflt=ht_id%3D204',
            'https://www.booking.com/searchresults.fr.html?label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB&sid=a8d69dac58d2b76c1d2bb743d5182840&aid=304142&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQHoAQH4AQuIAgGoAgO4Arm1mo0GwAIB0gIkZjE3NDZhZDItOTUyYS00NjljLWE5N2ItNzUxZjIxMTdjNDdi2AIG4AIB%3Bsid%3Da8d69dac58d2b76c1d2bb743d5182840%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-1412526%3Bclass_interval%3D1%3Bdest_id%3D-1410844%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D6f438386b3b60013%3Bss%3DBayonne%252C%2520Aquitaine%252C%2520France%3Bss_all%3D0%3Bss_raw%3DBayonne%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DBiarritz%3Bssne_untouched%3DBiarritz%3Btop_ufis%3D1%26%3B&ss=La+Rochelle%2C+Poitou-Charentes%2C+France&is_ski_area=&ssne=Bayonne&ssne_untouched=Bayonne&city=-1410844&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1&search_pageview_id=6f438386b3b60013&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1438604&dest_type=city&iata=LRH&place_id_lat=46.15911&place_id_lon=-1.15204&search_pageview_id=6f438386b3b60013&search_selected=true&ss_raw=La+Rochelle&nflt=ht_id%3D204'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        # filename
        city = response.url.split("/")[-1].replace(".fr.html", "")
        # hotels card
        hotels = response.css('div.sr__card.js-sr-card')
        for hotel in hotels:
            yield {
                'city' : city,
                'name': hotel.css('div.sr__card_content div.sr__card_main_row.bui-spacer--large div.sr__card_main header.bui-spacer--medium a h3.bui-spacer--smaller span.bui-card__title::text').get(),
                'url': "booking.com" + str(hotel.css('div.sr__card_content div.sr__card_main_row.bui-spacer--large div.sr__card_main header.bui-spacer--medium a::attr(href)').get()),
                # 'address': hotel.css('div.tags a.tag::text').getall(),
                # 'latitude': hotel.css('div.tags a.tag::text').getall(),
                # 'longitude': hotel.css('div.tags a.tag::text').getall(),
                'image_url': hotel.css('div.sr__card_photo a img::attr(src)').get(),
                'score': hotel.css('div.sr__card_content div.sr__card_main_row.bui-spacer--large div.sr__card_review div.bui-review-score.bui-review-score--end.bui-spacer--large div.bui-review-score__badge::text').get(),
                'description': hotel.css('div.sr__card_content div.sr__card_main_row.bui-spacer--large div.sr__card_main div.hotel-card__text.bui-spacer--medium p::text').get()
            }

    def parse_html(self, response):
        page = response.url.split("/")

        filename = response.url.split("/")[-1]
        with open(filename, 'wb') as f:
            f.write(response.body)
        self.log(f'Saved file {filename}')

In [19]:
filename = 'hotel.json'

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'src/' + filename : {"format": "json"},
    }
})

process.crawl(HotelsSpider)
process.start()

2022-04-21 13:12:39 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-04-21 13:12:39 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.9.7 (tags/v3.9.7:1016ef3, Aug 30 2021, 20:19:38) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1n  15 Mar 2022), cryptography 36.0.2, Platform Windows-10-10.0.19043-SP0
2022-04-21 13:12:39 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2022-04-21 13:12:39 [scrapy.extensions.telnet] INFO: Telnet Password: ea4747badde1dac8
2022-04-21 13:12:39 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-04-21 13:12:40 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewa

In [32]:
# read json from scrapy spider

# file to json
hotels_json = json.load(open(r'C:\Users\Riadh\Desktop\Certifications\Bloc_1\data\hotels_booking.json'))

# json to df
hotels_df = pd.json_normalize(hotels_json)

display(hotels_df.columns)

Index(['city', 'name', 'url', 'image_url', 'score', 'description'], dtype='object')

In [33]:
# clean some text
def clean_text(text) :
    return text.replace("\n", "")

hotels_df["name"] = hotels_df["name"].apply(lambda x : clean_text(x))
hotels_df["score"] = hotels_df["score"].apply(lambda x : clean_text(x))
hotels_df["description"] = hotels_df["description"].apply(lambda x : clean_text(x))

In [34]:
# astring score
hotels_df["score"] = hotels_df["score"].astype(float)

In [37]:
# save
export_hotels_csv = r"C:\Users\Riadh\Desktop\Certifications\Bloc_1\data\hotels_booking.csv"
hotels_df.to_csv(export_hotels_csv)

ETL & Datawarehouse

In [ ]:
# load dotenv variables
load_dotenv()
DBUSER = os.getenv('DBUSER')
DBPASS = os.getenv('DBPASS')
DBHOST = os.getenv('DBHOST')
DBNAME = os.getenv('DBNAME')

print(f"RDS Database : {DBNAME}")

In [ ]:
# engine : connected to our db
connection_string = f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}"
engine = create_engine(connection_string, echo=True, future=True)

In [ ]:
# sqlalchemy session to upload database
Session = sessionmaker(bind=engine)
local_session = Session()

In [ ]:
# declarative base
Base = declarative_base()

# create city table
# Let's define our table 
class City(Base):
    __tablename__ = "cities"

    uuid         = Column(String, primary_key=True)
    name         = Column(String)
    full_address = Column(String)
    latitude     = Column(Float)
    longitude    = Column(Float)

    def __repr__(self):
        return f"<City(name={self.name})>"

# weather table
class Weather(Base):
    __tablename__ = "weathers"

    weather_id    = Column(Integer, primary_key=True)
    city_uuid     = Column(String)
    volume_rain   = Column(Float)

    def __repr__(self):
        return f"<Weather(uuid={self.city_uuid}, volume={self.volume_rain})>"

# hotel table
class Hotel(Base):
    __tablename__ = "hotels"

    hotel_id     = Column(Integer, primary_key=True)
    name        = Column(String)
    city        = Column(String)
    url         = Column(String)
    image_url   = Column(String)
    score       = Column(Float)
    description = Column(String)

    def __repr__(self):
        return f"<Hotel(name={self.name}, score={self.score})>"

In [ ]:
# create_table
INIT = False
if INIT : 
    Base.metadata.create_all(engine)

Read Datalake files
weather_csv
hotels_booking_csv

In [ ]:
# s3 bucket
# dotenv variables
load_dotenv()
AWSS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')

# aws session
session = boto3.Session(aws_access_key_id=AWSS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

# s3 service
s3 = session.resource("s3")

# bucket 
bucket_name = 'projet_kayak'
kayak_bucket = s3.Bucket(bucket_name)

In [ ]:
# read weather data from s3
weather_key = "weather_data.csv"
weather_csv_obj = s3.Object(bucket_name, weather_key)
weather_csv = weather_csv_obj.get()['Body'].read().decode('utf-8') 

# dataframe
weather_df = pd.read_csv(weather_csv, index_col=[0])
display(weather_df.sample(2))
weather_df.columns

In [ ]:
# read hotels data from s3
hotels_key = "hotels_booking.csv"
hotels_key_obj = s3.Object(bucket_name, hotels_key)
hotels_csv = hotels_key_obj.get()['Body'].read().decode('utf-8') 

# dataframe
hotels_df = pd.read_csv(hotels_csv, index_col=[0])
display(hotels_df.sample(2))

ETL

In [38]:
# city df
keep_col = ['uuid', 'cities', 'full_address', 'latitude', 'longitude']
city_df = weather_df[keep_col].copy()

In [39]:
# weather df
keep_col = ['uuid', 'volume_rain_7days']
weather_df_volume = weather_df[keep_col].copy()
weather_df_volume = weather_df_volume.reset_index()
display(weather_df_volume.sample(2))

,index,uuid,volume_rain_7days
25,25,1b2a51a0-3143-4458-affe-7f4fdcad7ff2,0.929
0,0,3a80f21a-ebcd-4693-ae1c-a11f12f445cc,3.364


In [40]:
# hotel dataframe
hotels_df = hotels_df.reset_index().copy()
display(hotels_df.sample(2))

,index,city,name,url,image_url,score,description
245,245,carcassonne,Mercure Carcassonne Porte De La Cité,booking.com/hotel/fr/carcassonne-porte-de-la-c...,https://cf.bstatic.com/xdata/images/hotel/squa...,8.2,"Offrant une vue sur les remparts de la ville, ..."
291,291,bayonne,Hotel Cote Basque,booking.com/hotel/fr/cote-basque.fr.html?sid=d...,https://cf.bstatic.com/xdata/images/hotel/squa...,8.0,L'Hotel Cote Basque est situé au cœur de la vi...


In [51]:
hotels_df.columns

Index(['index', 'city', 'name', 'url', 'image_url', 'score', 'description'], dtype='object')

In [49]:
hotels_df.columns
hotelstop = hotels_df[['name','score']]
hotelstop20= hotelstop.head(20)
hotelstop20.to_csv('Top20HOTELS.csv')

In [50]:
fig = px.scatter_geo(hotelstop20,lat='lat',lon='long', hover_name="id")
fig.update_layout(title = 'World map', title_x=0.5)
fig.show

DataWarehouse
Tables :

cities
weathers
hotels

In [41]:
# fill City TABLE
for idx in range(city_df.shape[0]) : 
    # get row
    city_row = city_df.iloc[idx]

    # create city object
    city_uuid         = city_row["uuid"]
    city_name         = city_row["cities"]
    city_fulladdress  = city_row["full_address"]
    city_latitude     = city_row["latitude"]
    city_longitude    = city_row["longitude"]
    city = City(uuid=city_uuid, name=city_name, full_address=city_fulladdress,
                latitude=city_latitude, longitude=city_longitude)

    # add & commit
    # verify if cities in table
    city_in_table = local_session.query(City).filter_by(name=city_name).first()

    if not city_in_table : 
        # Add values to db 
        local_session.add(city)

        # Commit the results 
        local_session.commit()   
        
    print(city)

NameError: name 'City' is not defined

In [ ]:
# fill Weather TABLE
for idx in range(weather_df_volume.shape[0]) : 
    # get row
    weather_row = weather_df_volume.iloc[idx]

    # create city object
    weather_id        = int(weather_row["index"])
    city_uuid         = weather_row["uuid"]
    weather_volume    = float(weather_row["volume_rain_7days"])
    weather = Weather(weather_id=weather_id, city_uuid=city_uuid, volume_rain=weather_volume)

    # add & commit
    # verify if cities in table
    weather_in_table = local_session.query(Weather).filter_by(weather_id=weather_id).first()

    if not weather_in_table : 
        # Add values to db 
        local_session.add(weather)

        # Commit the results 
        local_session.commit()   
        
    print(weather)

In [ ]:
# fill Hotel TABLE
for idx in range(hotels_df.shape[0]) : 
    # get row
    hotel_row = hotels_df.iloc[idx]

    # create city object
    hotel_id        = int(hotel_row["index"])
    name            = hotel_row["name"]
    city            = hotel_row["city"]
    url             = hotel_row["url"]
    image_url       = hotel_row["image_url"]
    score           = hotel_row["score"]
    description     = hotel_row["description"]

    # hotel object
    hotel = Hotel(hotel_id=hotel_id, name=name, 
                    city=city, url=url, 
                    image_url=image_url, score=score, 
                    description=description)

    # add & commit
    # verify if cities in table
    hotel_in_table = local_session.query(Hotel).filter_by(name=name).first()

    if not hotel_in_table : 
        # Add values to db 
        local_session.add(hotel)

        # Commit the results 
        local_session.commit()   
        
    print(hotel)

In [67]:
df = pd.read_csv("C:/Users/Riadh\Desktop/Certifications/Bloc_1/data/TOP20HOTELS.csv", sep=';', encoding='latin-1')

df.head()


,Column1,city,Latitude,Longitude,name,score
0,206,bormes-les-mimosas,43.151682,6.342202,HomeStay Bormes,9.7
1,226,aigues-mortes,43.567172,4.192587,Boutique Hôtel des Remparts & Spa,9.6
2,178,uzes,44.012110,4.419946,Le trente trois,9.2
3,50,strasbourg,48.573405,7.752111,Hôtel ORIGAMI,9.1
4,94,eguisheim,48.043265,7.306522,L'Hostellerie du Château,9.1


In [85]:
fig = px.scatter_mapbox(
    df,  # Our DataFrame
    lat = "Latitude",
    lon = "Longitude",
    #center = {"lat": 2.43, "lon": 49.13},  # where map will be centered
    width = 600,  # Width of map
    height = 600,  # Height of map
    hover_data = ["score"],  # what to display when hovering mouse over coordinate
)

fig.update_layout(mapbox_style="open-street-map") # adding beautiful street layout to map

fig.show()

In [86]:


fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", color="score", size="score", zoom=10, mapbox_style="carto-positron")
fig.show()

In [94]:
fig = px.scatter_mapbox(
    top_5_destinations,  # Our DataFrame
    lat = "latitude",
    lon = "longitude",
    #center = {"lat": 2.43, "lon": 49.13},  # where map will be centered
    width = 600,  # Width of map
    height = 600,  # Height of map
    hover_data = ["volume_rain_7days"],  # what to display when hovering mouse over coordinate
)

fig.update_layout(mapbox_style="open-street-map") # adding beautiful street layout to map

fig.show()